In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
# root = "/media/nick/hdd021/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"

# path to save data
out_path = os.path.join(root, "analyses", "crossmodal", "hotfish", "")
os.makedirs(out_path, exist_ok=True)

# path to figures and data
fig_path = os.path.join(root, "figures", "crossmodal", "hotfish", "")
os.makedirs(fig_path, exist_ok=True)

In [ ]:
stage_list = [0, 6, 8, 10, 14, 24, 48]
n_cell_list = [1, 9e3, 11e3, 20e3, 25e3, 40e3, 3e5]
cell_count_df = pd.DataFrame(stage_list, columns=["stage_hpf"])
cell_count_df["n_cells"] = n_cell_list

fig = px.scatter(cell_count_df, x="stage_hpf", y="n_cells", trendline="lowess", log_y=True,)
fig.show()

In [ ]:
from scipy.optimize import least_squares

def trend_fn(params, time=np.asarray(stage_list)):
    trend_pd = params[0] * (1 - np.exp(-params[1] * time))
    return trend_pd


def loss_fn(params, trend=np.log10(np.asarray(n_cell_list))):
    ls = trend - trend_fn(params)
    return ls



In [ ]:
res_2 = least_squares(loss_fn, x0=[6, 0.01], bounds=([np.log10(3e5), 0], [7, 1]))

In [ ]:
res_2.x

In [ ]:
import plotly.graph_objects as go

stage_vec_pd = np.linspace(np.min(stage_list), np.max(stage_list), 100)
log_trend_pd = trend_fn(params=res_2.x, time=stage_vec_pd)
trend_pd = 10**(log_trend_pd)

fig = px.scatter(cell_count_df, x="stage_hpf", y="n_cells", log_y=True,)
fig.add_traces(go.Scatter(x=stage_vec_pd, y=trend_pd))
fig.show()

In [ ]:
out_table = pd.DataFrame(stage_vec_pd, columns=["stage_hpf"])
out_table["n_cells"] = trend_pd
out_table["n_cells_log"] = log_trend_pd
out_table.to_csv(os.path.join(out_path, "n_cell_table.csv"), index=False)